In [2]:
Q = 1234567891011
x = 25

In [3]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(-770757049682, 1140152674111, 865172266607)

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(25)

In [7]:
decrypt(a, b, c)

25

In [8]:
decrypt(a, b)

569855822200

In [9]:
x = encrypt(25)     # x是一个列表
y = encrypt(5)      # y是一个列表

In [10]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [11]:
decrypt(*add(x,y))

30

+ Step 3: SMPC Using PySyft
    + addition
    + multiplication
    + comparison

In [12]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

In [13]:
x = torch.tensor([25])
x

tensor([25])

In [14]:
encrypted_x = x.share(bob, alice, bill)     # 利用share()函数进行加密
encrypted_x

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:90932448612 -> bob:3435392766]
	-> [PointerTensor | me:39337888085 -> alice:82731405177]
	-> [PointerTensor | me:29516718754 -> bill:8946126755]
	*crypto provider: me*

In [15]:
encrypted_x.get()       # 利用get()函数进行解密

tensor([25])

In [16]:
list(bob._tensors.values())

[]

In [17]:
x = torch.tensor([25]).share(bob, alice, bill)

In [18]:
# Bob's share
bobs_share = list(bob._tensors.values())[0]
bobs_share

tensor([3168853688138303006])

In [19]:
# Alice's share
alices_share = list(alice._tensors.values())[0]
alices_share

tensor([4138039501719597150])

In [20]:
# Bill's share
bills_share = list(bill._tensors.values())[0]
bills_share

tensor([-7306893189857900131])

In [21]:
(bobs_share + alices_share + bills_share)

tensor([25])

In [22]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [23]:
z = x + y
z

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:30086260427 -> bob:44055332809]
	-> [PointerTensor | me:82741720398 -> alice:84971375145]
	*crypto provider: me*

In [24]:
z.get()

tensor([30])

In [25]:
z = x - y
z.get()

tensor([20])

In [26]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [27]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [28]:
# multiplication

z = x * y
z.get()

tensor([125])

In [29]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [30]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

In [31]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [32]:
z = x > y
z.get()

tensor([1])

In [33]:
z = x <= y
z.get()

tensor([0])

In [34]:
z = (x == y)
z.get()

tensor([0])

In [35]:
z = ((x == y) + 20)
z.get()

tensor([20])

In [37]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:22268840830 -> bob:72604516262]
	-> [PointerTensor | me:16725134435 -> alice:66882961925]
	*crypto provider: crypto_provider*

In [39]:
x_max = x.max()
x_max.get()

RuntimeError: shape '[3]' is invalid for input of size 12

In [40]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values = x.max(dim=0)       # 每行的最大值
max_values.get()

RuntimeError: shape '[1]' is invalid for input of size 4